In [1]:
import os
import textwrap
import torch
import chromadb
import langchain
import openai
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader, YoutubeLoader
from langchain.embeddings import HuggingFaceEmbeddings,HuggingFaceInstructEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.document_loaders.image import UnstructuredImageLoader

In [2]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [3]:
from langchain_community.document_loaders import UnstructuredPDFLoader
# convert_from_path('a_file_name.pdf', poppler_path='poppler-24.02.0\Library\bin')
pdf_loader = UnstructuredPDFLoader('G:\youtube_transcript\\resume-sample.pdf')

In [4]:
from langchain_community.document_loaders import UnstructuredPDFLoader
pdf_pages = pdf_loader.load_and_split()

In [ ]:
# from pdf2image import convert_from_path
# pages = convert_from_path(r"G:\Resume_Project\resume_juanjosecarin.pdf", poppler_path=r'C:\Program Files (x86)\poppler-24.02.0\Library\bin')
# pages

In [5]:
len(pdf_pages)

7

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=512)
texts = text_splitter.split_documents(pdf_pages)
len(texts)

19

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings, HuggingFaceInstructEmbeddings

In [11]:
model_name = "BAAI/bge-small-en-v1.5"

hf_embeddings = HuggingFaceEmbeddings(
    model_name = model_name)

In [12]:
db = Chroma.from_documents(texts, hf_embeddings)

In [23]:
llm = Ollama(model='llama3:8b')

In [14]:
def generate_prompt(prompt, sys_prompt):
    return f"[INST] <<SYS>> {sys_prompt} <</SYS>> {prompt} [/INST]"

In [15]:
sys_prompt = "Use following piece of context to answer the question in less than 20 words"
template = generate_prompt(
    """
    {context}

    Question : {question}
    """
    , sys_prompt)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [17]:
chain_type_kwargs = {"prompt": prompt}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs,
)

In [ ]:
result = qa_chain("what projects candidate worked on ?")

In [ ]:
result = qa_chain("where did candidate study?")

In [ ]:
result = qa_chain("what skillset candidate has?")

In [ ]:
result = qa_chain("what's the contact detail of candidate?")

In [ ]:
result = qa_chain("how many years of work experience candidate has?")

In [ ]:
result = qa_chain("In which companies with what jobtitle did candidate work?")